### Liberaries Importing

In [ ]:
import pandas as pd
import numpy as np

### Data Importing

In [ ]:
df = pd.read_csv('user_data_unlceaned.csv',index_col=0)
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.shape

### Data Cleaning

In [ ]:
df.info()

The first thing can be noticed from the previous output is that the `Average Monthly Spend on Entertainment` data type is object when it should be integer.
The second thing is the null values in the `lifestyle choice` column

In [ ]:
#exploring the Average Monthly Spend on Entertainment column
df['Average Monthly Spend on Entertainment'].head()

OK, We found the issue.
The currency sign beside the number is the reason why the column's data type is `object`.
lets get rid of it.

In [ ]:
# firstly, let's make sure that there is no other currancy sign except the $ sign
currancy_sign = r'[€£¥₹¢]'
other_currancy = df['Average Monthly Spend on Entertainment'].str.contains(currancy_sign)
other_currancy.sum()

In [ ]:
# removing the $ sign from the `Average Monthly Spend on Entertainment` column
df['Average Monthly Spend on Entertainment'] = df['Average Monthly Spend on Entertainment'].str.replace('$',"").astype(float)

In [ ]:
df['Average Monthly Spend on Entertainment'].head()

In [ ]:
#now, let's work on `Lifestyle Choice` column
#first, we need to count the null values
df['Lifestyle Choice'].isna().sum()

In [ ]:
# The 9 values are not significant enough to impact the dataset, so we will drop them.
df.dropna(inplace=True)

In [ ]:
df.info()

In [ ]:
#now, lets explore the other columns
df.head(10)

I've noticed inconsistencies in the Gender column, with some values represented as ('M', 'F') and others as ('female', 'male').  so let's unify these.

In [ ]:
#check value counts in the gender columns
df['Gender'].value_counts()

In [ ]:
df['Gender'] = df['Gender'].str.replace('female','F').replace('male','M')

In [ ]:
df['Gender'].value_counts()

In [ ]:
#now, let's check if there is any duplicate in the dataset

df.duplicated().sum()

No duplicates

In [ ]:
#checking duplicates amonge [['First Name','Last Name', 'Age', 'Gender']] columns

df[['First Name','Last Name', 'Age', 'Gender']].duplicated().sum()

We need to check if these duplications are for the same people or just by chance, we going to check this by adding the `Location` column to the duplication check function.

In [ ]:
df[['First Name','Last Name', 'Age', 'Gender','Location']].duplicated().sum()

Ok, so we are gonna drop these 13 records to reduce redundancy as possible.

In [ ]:
df.drop_duplicates(subset=['First Name', 'Last Name', 'Age', 'Gender', 'Location'], inplace=True)


After investigating the data more, we found that `Location` is a multivalue column. So, we need to expand this column to make the data exploration, aggregation, analysis more eaiser.

In [ ]:
#creating City, State, Country from the Location dataset

df[['City', 'State', 'Country']]  = df['Location'].str.lstrip().str.split(',',n=2, expand=True)

In [ ]:
df.columns

Now, we can drop `Location` column to prevent redundancy.


In [ ]:
#drop Location column
df.drop(columns=['Location'],inplace=True)

In [ ]:
#Rounding ['Number of Charity Donations in Last Year','Number of Professional Trainings Attended']

r_col = ['Number of Charity Donations in Last Year','Number of Professional Trainings Attended']
df[r_col] = df[r_col].round().astype(int)


In [ ]:
df.info()

### DataBase Normalization

Our data after cleaning is 32-column, so it's worth cosidering normalization to facilitate the upcoming data analysis and visulization, as well as, making storing data in a rational database easier.
So we are gonna expand the `df` into 3 tables one for `Location` entity, the other for `User_details` entity, and the last one is our main table which is `Lifestyles`.

In [ ]:
#creating Location_df

location_df = df[['City','State','Country']].drop_duplicates()
location_df['Location_ID'] = location_df.index+1
location_df.head()

In [ ]:
#add Location_id to our main table to use it as a foriegn key

lifestyle_df = df.merge(location_df,on=['City','State','Country'])

In [ ]:
#Checking df columns
lifestyle_df.columns

In [ ]:
#Now we can delete location deletes from the main table
lifestyle_df.drop(columns = ['City','State','Country'],inplace=True)

In [ ]:
lifestyle_df.shape

In [ ]:
#creating User_details table
user_details = lifestyle_df[['First Name','Last Name', 'Age', 'Gender','Location_ID']]
user_details = user_details.copy()
user_details['User_ID'] = user_details.index + 1
user_details.head()

In [ ]:
#add User_id to our main table to use it as a foriegn key

lifestyle_df = lifestyle_df.merge(user_details,on=['First Name','Last Name', 'Age', 'Gender','Location_ID'])

In [ ]:
lifestyle_df.columns

In [ ]:
lifestyle_df.drop(columns = ['First Name','Last Name', 'Age', 'Gender','Location_ID'],inplace=True)

In [ ]:
lifestyle_df.columns

### Date Exporting

In [ ]:
df.to_csv('user_data_CLEANED.csv')

In [ ]:
lifestyle_df.to_csv('lifestyle_df.csv')

In [ ]:
location_df.to_csv('location_df.csv', index=False)

In [ ]:
user_details.to_csv('user_details.csv', index=False)